In [96]:
import pandas as pd
import geopandas as gpd
import os
from geopy import distance
import calendar

os.chdir("/Users/anorawu/Documents/GitHub/CloudSeeding/data/炮台数据")

In [97]:
fort_data = pd.read_csv("processed_cloud_data.csv")
fort_data = fort_data[fort_data['year']<=2022]

lon_max = fort_data['longitude'].max() + 1
lon_min = fort_data['longitude'].min() - 1
lat_max = fort_data['latitude'].max() + 1
lat_min = fort_data['latitude'].min() - 1

In [98]:
fort_points = fort_data[['index','longitude','latitude']].drop_duplicates()

In [99]:
rain_data = pd.read_stata("weatherstation_2010_2022.dta")
rain_data = rain_data.dropna(subset=['Lat', 'Lon'])
rain_data = rain_data[(rain_data['Lat'] >= lat_min) & (rain_data['Lat'] <= lat_max)]
rain_data = rain_data[(rain_data['Lon'] >= lon_min) & (rain_data['Lon'] <= lon_max)]

rain_points = rain_data[['StationId','Lon','Lat']].drop_duplicates()

In [100]:
rain_gdf = gpd.GeoDataFrame(
    rain_points,
    geometry=gpd.points_from_xy(rain_points['Lon'], rain_points['Lat']),
    crs="EPSG:4326" 
)

In [101]:
fort_gdf = gpd.GeoDataFrame(
    fort_points,
    geometry=gpd.points_from_xy(fort_points['longitude'], fort_points['latitude']),
    crs="EPSG:4326" 
)

In [102]:
radius = 0.5
dic = {}

for _, fort_point in fort_gdf.iterrows():  
    buffer = fort_point.geometry.buffer(radius)  
    
    pts_inside = rain_gdf[rain_gdf.within(buffer)]
    
    pts_dic = {}
    for _, row in pts_inside.iterrows():
        dist = distance.distance(
            (fort_point.geometry.y, fort_point.geometry.x),  
            (row['Lat'], row['Lon'])                        
        ).km
        pts_dic[row['StationId']] = dist
    
    sorted_pts = sorted(pts_dic.items(), key=lambda item: item[1])[:4]
    
    dic[fort_point['index']] = sorted_pts  



In [ ]:
columns = ['Alti', 'EVP', 'GST_Avg', 'GST_Max', 'GST_Min', 'PRE_Max_1h',
       'PRE_Time_2008', 'PRE_Time_0820', 'PRE_Time_2020', 'PRS_Avg', 'PRS_Max',
       'PRS_Min', 'RHU_Avg', 'RHU_Min', 'SSH', 'TEM_Avg', 'TEM_Max', 'TEM_Min',
       'WIN_S_2mi_Avg', 'WIN_S_10mi_Avg', 'WIN_D_S_Max', 'WIN_S_Max',
       'WIN_D_INST_Max', 'WIN_S_Inst_Max']

for year in range(2011,2023):
    for month in range(1,13):
        days_in_month = calendar.monthrange(year, month)[1]
        for day in range(1,days_in_month+1):
            for idx, fort_point in fort_points:
                pts_list = dic[fort_point['index']]

                weight_sum = 0
                list_of_data = []
                for pts in pts_list:
                    rain_id,dist = pts
                    # Make a copy of the filtered rows
                    filtered_row = rain_data[
                        (rain_data['Year'] == year) & 
                        (rain_data['Mon'] == month) & 
                        (rain_data['Day'] == day) & 
                        (rain_data['StationId'] == rain_id)
                    ].copy()  

                    filtered_row[columns] = filtered_row[columns] * 1 / (dist**2)
                    list_of_data.append(filtered_row)
                    weight_sum += 1/(dist**2) 
                
                concat_data = pd.concat(list_of_data)
                concat_data_sum = concat_data[columns].sum(min_count=4)
                    
                    

In [114]:
import pandas as pd
import numpy as np

# Generate a random DataFrame
np.random.seed(42)  # For reproducibility

# Create random data
n_rows = 10
data = {
    'Year': np.random.randint(2011, 2023, size=n_rows),
    'Mon': np.random.randint(1, 13, size=n_rows),
    'Day': np.random.randint(1, 29, size=n_rows),  # Assume 28 days max for simplicity
    'Lat': np.random.uniform(-90, 90, size=n_rows),
    'Lon': np.random.uniform(-180, 180, size=n_rows),
    'Rainfall': np.random.uniform(0, 100, size=n_rows),  # Random rainfall data
    'Temperature': np.random.uniform(-30, 50, size=n_rows),  # Random temperature data
}

# Create DataFrame
test = pd.DataFrame(data)
concat_test = pd.concat([test,test])
concat_test


,Year,Mon,Day,Lat,Lon,Rainfall,Temperature
0,2017,11,24,-88.728065,173.963119,23.089383,-13.364667
1,2014,8,12,-85.848763,-11.965358,24.102547,15.416026
2,2021,5,6,4.459439,129.578546,68.326352,-27.494937
3,2018,4,2,-18.025025,64.910714,60.999666,37.382782
4,2015,8,28,-81.600181,-17.820269,83.319491,5.980331
5,2017,8,21,85.275993,-175.224614,17.336465,1.612019
6,2020,3,1,-48.101159,159.192632,39.106061,44.132709
7,2013,6,12,-73.690842,22.783758,18.223609,28.181760
8,2017,5,26,21.309482,-41.250059,75.536141,-3.876738
9,2021,2,22,-21.156842,-174.252149,42.515587,15.635518


In [110]:
rain_data.columns
            

Index(['StationId', 'Lat', 'Lon', 'lat_mode', 'lon_mode', 'Alti', 'Year',
       'Mon', 'Day', 'EVP', 'GST_Avg', 'GST_Max', 'GST_Min', 'PRE_Max_1h',
       'PRE_Time_2008', 'PRE_Time_0820', 'PRE_Time_2020', 'PRS_Avg', 'PRS_Max',
       'PRS_Min', 'RHU_Avg', 'RHU_Min', 'SSH', 'TEM_Avg', 'TEM_Max', 'TEM_Min',
       'WIN_S_2mi_Avg', 'WIN_S_10mi_Avg', 'WIN_D_S_Max', 'WIN_S_Max',
       'WIN_D_INST_Max', 'WIN_S_Inst_Max'],
      dtype='object')